In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## ICTV VMR

- VMR_MSL39_v1 contains 528 additional isolates (509 more species) that infect vertebrates and/or invertebrates compared to VMR_MSL38_v2 that we used for our analysis.

In [2]:
file_vmr = "~/1_dataset_preparation/msl38v2/VMR_MSL38_v2.xlsx"
df_vmr = pd.read_excel(file_vmr)

In [3]:
df_vmr

,Sort,Isolate Sort,Realm,Subrealm,Kingdom,Subkingdom,Phylum,Subphylum,Class,Subclass,...,Species,Exemplar or additional isolate,Virus name(s),Virus name abbreviation(s),Virus isolate designation,Virus GENBANK accession,Virus REFSEQ accession,Genome coverage,Genome composition,Host source
0,1,1,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,Tokiviricetes,NaN,...,Alphalipothrixvirus SBFV2,E,Sulfolobales Beppu filamentous virus 2,SBFV2,SBFV2 clone B,MK064563,NC_048128,Coding-complete genome,dsDNA,archaea
1,2,1,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,Tokiviricetes,NaN,...,Alphalipothrixvirus SFV1,E,Sulfolobus filamentous virus 1,SFV1,SFV1 isolate S48,MH447526,NC_048037,Complete genome,dsDNA,archaea
2,3,1,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,Tokiviricetes,NaN,...,Acidianus filamentous virus 3,E,Acidianus filamentous virus 3,AFV3,NaN,AM087120,NC_010155,Complete genome,dsDNA,archaea
3,4,1,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,Tokiviricetes,NaN,...,Acidianus filamentous virus 6,E,Acidianus filamentous virus 6,AFV6,NaN,AM087121,NC_010152,Complete genome,dsDNA,archaea
4,5,1,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,Tokiviricetes,NaN,...,Acidianus filamentous virus 7,E,Acidianus filamentous virus 7,AFV7,NaN,AM087122,NC_010153,Complete genome,dsDNA,archaea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12948,12949,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tomato leaf curl deltasatellite,E,tomato leaf curl deltasatellite,ToLCD,AU/96,U74627,NC_002743,Complete genome,ssDNA,plants
12949,12950,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tomato yellow leaf distortion deltasatellite 1,E,tomato yellow leaf distortion deltasatellite 1,ToYLDD1,CU/404N1/10,JN819495,NC_043204,Complete genome,ssDNA,plants
12950,12951,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tomato yellow leaf distortion deltasatellite 2,E,tomato yellow leaf distortion deltasatellite 2,ToYLDD2,CU/603N1/11,KU232893,NC_043205,Complete genome,ssDNA,plants
12951,12952,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Heterocapsa circularisquama DNA virus 01,E,Heterocapsa circularisquama DNA virus 01,HcDNAV01,NaN,AB522601,NC_038702,Partial genome,dsDNA,algae


In [46]:
df_animals = df_vmr[(df_vmr['Host source'] == 'vertebrates') |
                     (df_vmr['Host source'] == 'invertebrates') |
                     (df_vmr['Host source'] == 'invertebrates, vertebrates')]

In [47]:
# make a subset of the df_animals with 'Species' and 'Virus GENBANK accession' columns
df_animals_subset = df_animals[['Sort', 'Species', 'Virus GENBANK accession', 'Genome composition']]

In [48]:
df_animals_subset.dropna(subset=['Virus GENBANK accession'], inplace=True)
# reset the index
df_animals_subset.reset_index(drop=True, inplace=True)

/tmp/ipykernel_4155020/3859116041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_animals_subset.dropna(subset=['Virus GENBANK accession'], inplace=True)


In [49]:
# make a new column for genbank ids and count the number of segments
for index, row in df_animals_subset.iterrows():
    
    genbank_str = row['Virus GENBANK accession']
    
    if ';' in genbank_str:
        
        genbank_list = genbank_str.split('; ')
        genbank_list_new = []
        
        for genbank_id in genbank_list:
            
            if ':' in genbank_id:
                genbank_id = genbank_id.split(': ')[1]
                genbank_list_new.append(genbank_id)
            else:
                genbank_list_new.append(genbank_id)
        
        genbank_str_size = len(genbank_list_new)
        genbank_str_new = '|'.join(genbank_list_new)

    else:
        genbank_str_new = genbank_str
        genbank_str_size = 1

    #print(genbank_str)
    #print(genbank_str_new)

    df_animals_subset.loc[index, 'GENBANK accession'] = genbank_str_new
    df_animals_subset.loc[index, 'segment_count'] = genbank_str_size

/tmp/ipykernel_4155020/561793997.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_animals_subset.loc[index, 'GENBANK accession'] = genbank_str_new
/tmp/ipykernel_4155020/561793997.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_animals_subset.loc[index, 'segment_count'] = genbank_str_size


In [50]:
df_animals_subset['segment_count'] = df_animals_subset['segment_count'].astype(int)
df_animals_subset.drop(columns='Virus GENBANK accession', inplace=True)
df_animals_subset.sort_values('segment_count', ascending=False, inplace=True)
df_animals_subset.reset_index(drop=True, inplace=True)

/tmp/ipykernel_4155020/3725647293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_animals_subset['segment_count'] = df_animals_subset['segment_count'].astype(int)
/tmp/ipykernel_4155020/3725647293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_animals_subset.drop(columns='Virus GENBANK accession', inplace=True)
/tmp/ipykernel_4155020/3725647293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [51]:
df_animals_subset

,Sort,Species,Genome composition,GENBANK accession,segment_count
0,12760,Ichnoviriform fumiferanae,dsDNA,AB289903|AB289904|AB289905|AB289906|AB289907|A...,105
1,12759,Ichnoviriform fugitivi,dsDNA,AY556383|AB291165|AB291166|AY935249|AB291167|A...,56
2,12723,Bracoviriform congregatae,dsDNA,AJ632304|AJ632305|AJ632306|AJ632307|AJ632308|A...,30
3,12729,Bracoviriform facetosae,dsDNA,EU001285|EU001284|EU001283|EU001282|EU001281|E...,29
4,12737,Bracoviriform indiense,dsDNA,EF051512|EF051511|EF051510|EF051509|EF051508|E...,27
...,...,...,...,...,...
4402,8906,Ephemerovirus febris,ssRNA(-),AF234533,1
4403,8907,Ephemerovirus guangdong,ssRNA(-),OK086698,1
4404,8908,Ephemerovirus hayes,ssRNA(-),MH507506,1
4405,8909,Ephemerovirus henan,ssRNA(-),OK086697,1


In [52]:
file_all = "~/1_dataset_preparation/virosphere-fold-v1_fulldata.csv"
df_all = pd.read_csv(file_all)

In [53]:
df_all

,header,translation,gene,product,protein_id,mat_pept_id,reg_id,uniq_id,pept_cat,note,...,Exemplar or additional isolate,Virus name(s),Virus name abbreviation(s),Virus isolate designation,Virus REFSEQ accession,Genome coverage,Genome composition,Host source,host,coordinates
0,CAA52564.1_4563,M,E2,early protein,CAA52564.1,NaN,NaN,CAA52564.1,protein,NaN,...,A,human papillomavirus 35,HPV35,NaN,NaN,Complete genome,dsDNA,vertebrates,NaN,[2713:>2717](+)
1,CAA27332.1.11_7160,G,NaN,polyprotein precursor,CAA27332.1,CAA27332.1.11,NaN,CAA27332.1.11,mat_pept,putative replicase component NS5,...,E,yellow fever virus,YFV,17D,NC_002031,Complete genome,ssRNA(+),"invertebrates, vertebrates",NaN,[2506:>2507]
2,AAA47931.1.5_10363,NPAA,NaN,polyprotein,AAA47931.1,AAA47931.1.5,NaN,AAA47931.1.5,mat_pept,unnamed,...,A,cardiovirus B2; Vilyuisk human encephalomyelit...,CaV-B2; VHEV,V1 (human/Russia-Siberia/1955),NaN,Partial genome,ssRNA(+),vertebrates,NaN,[926:930]
3,CAA27339.1.3_11752,TNRLTH,NaN,NaN,CAA27339.1,CAA27339.1.3,NaN,CAA27339.1.3,mat_pept,pol protein (6 aa),...,E,Hardy-Zuckerman feline sarcoma virus,HZFeSV,NaN,NC_038923,Partial genome,ssRNA-RT,vertebrates,NaN,[784:790]
4,CAI74981.1.3_11505,ASGRALA,NaN,polyprotein precursor,CAI74981.1,CAI74981.1.3,NaN,CAI74981.1.3,mat_pept,pep7 protein,...,E,Tellina virus 1,TV1,NaN,SegA: NC_038869; SegB: NC_038870,Complete genome,dsRNA,invertebrates,Tellina tenuis,[492:499]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87135,ASO76148.1_10247,MVLIVPLSRTNRPPITKPRHFHALPNYYTKYGSDIIKRNIYYNSVD...,ORF1ab,ORF1ab,ASO76148.1,NaN,NaN,ASO76148.1,protein,pp1ab replicase polyprotein,...,E,Morelia viridis nidovirus,MVNV,NaN,NC_035465,Complete genome,ssRNA(+),vertebrates,Morelia viridis,"join{[855:17505](+), [17504:24440](+)}"
87136,QRC47047.1_10244,MVFIHPNFVYLSKGLGKPRCTASTIVQREYQYQYISYPHKTVGSIF...,ORF1AB,ORF1AB,QRC47047.1,NaN,NaN,QRC47047.1,protein,ribosomal slippery sequence AAAAAAC at 17563.....,...,E,Veiled chameleon serpentovirus A,VCSTV-A,A,NaN,Complete genome,ssRNA(+),vertebrates,Chamaeleo calyptratus,"join{[409:17569](+), [17568:24540](+)}"
87137,AIJ50565.1_10246,MVLIVPLAYTNRPPICKPRRYSVLPSYHHKYGSDMIKRNIYYRGTD...,ORF1AB,pp1ab,AIJ50565.1,NaN,NaN,AIJ50565.1,protein,pp1ab replicase polyprotein,...,E,ball python nidovirus,BPNV,19541,NC_024709,Complete genome,ssRNA(+),vertebrates,Python regius,"join{[1017:18411](+), [18410:25343](+)}"
87138,QRC47039.1_10254,MVVLQVVAQNATKSGETPGWRLPLKSLEKGKAIARKSMGSSLTLTQ...,ORF1AB,ORF1AB,QRC47039.1,NaN,NaN,QRC47039.1,protein,ribosomal slippery sequence AAAAAAC at 20100.....,...,E,Veiled chameleon serpentovirus B,VCSTV-B,B,NaN,Complete genome,ssRNA(+),vertebrates,Chamaeleo calyptratus,"join{[399:20106](+), [20105:27182](+)}"


In [4]:
file_predicted = "~/2_protein_structure_prediction/virosphere-fold-v1_predicted_dataset.csv"
df_predicted = pd.read_csv(file_predicted)

In [39]:
len(df_predicted['Sort'].unique())

4330

In [ ]:
n = 0

for index, row in df_animals_subset.iterrows():

    # get the GENBANK accession and the number of coding segments
    sort_code = row['Sort']
    acc_str = row['GENBANK accession']
    coding_seqment_number = row['segment_count']

    # initialize the total protein, peptide and region information
    total_protein_number = 0
    total_predicted_protein_number = 0
    total_peptide_number = 0
    total_region_number = 0

    # initialize the total protein, peptide and region residue number
    total_protein_residue_number = 0
    total_predicted_protein_residue_number = 0
    total_peptide_residue_number = 0
    total_region_residue_number = 0
    
    # convert the string of GENBANK accessions into a list
    if '|' in acc_str:
        acc_list = acc_str.split('|')
    else:
        acc_list = [acc_str]
    
    # iterate over the list of GENBANK accessions and get the protein, peptide and region information
    for acc in acc_list:
        
        # get the protein information
        df_all_protein = df_all[(df_all['acc'] == acc) & (df_all['pept_cat'] == 'protein')]
        protein_number = df_all_protein.shape[0]
        residue_number = df_all_protein['protlen'].sum()
        # if there are no proteins, then the genome segment is not a coding segment
        if protein_number == 0:
            coding_seqment_number -= 1
        # add the protein information to the total
        total_protein_number += protein_number
        total_protein_residue_number += residue_number

        # get the predicted protein information
        df_predicted_protein = df_predicted[(df_predicted['acc'] == acc) & (df_predicted['pept_cat'] == 'protein')]
        predicted_protein_number = df_predicted_protein.shape[0]
        predicted_residue_number = df_predicted_protein['protlen'].sum()
        # add the predicted protein information to the total
        total_predicted_protein_number += predicted_protein_number
        total_predicted_protein_residue_number += predicted_residue_number

        # get the peptide information
        df_all_peptide = df_all[(df_all['acc'] == acc) & (df_all['pept_cat'] == 'mat_pept')]
        peptide_number = df_all_peptide.shape[0]
        residue_number = df_all_peptide['protlen'].sum()
        # add the peptide information to the total
        total_peptide_number += peptide_number
        total_peptide_residue_number += residue_number

        # get the region information
        df_all_region = df_all[(df_all['acc'] == acc) & (df_all['pept_cat'] == 'region')]
        region_number = df_all_region.shape[0]
        residue_number = df_all_region['protlen'].sum()
        # add the region information to the total
        total_region_number += region_number
        total_region_residue_number += residue_number

    # add the total protein information to the df_animals_subset
    df_animals_subset.loc[index, 'coding_segment_count'] = coding_seqment_number
    df_animals_subset.loc[index, 'protein_number'] = total_protein_number
    df_animals_subset.loc[index, 'protein_residue_number'] = total_protein_residue_number
    # add the total predicted protein information to the df_animals_subset
    df_animals_subset.loc[index, 'predicted_protein_number'] = total_predicted_protein_number
    df_animals_subset.loc[index, 'predicted_protein_residue_number'] = total_predicted_protein_residue_number
    # add the total peptide information to the df_animals_subset
    df_animals_subset.loc[index, 'peptide_number'] = total_peptide_number
    df_animals_subset.loc[index, 'peptide_residue_number'] = total_peptide_residue_number
    # add the total region information to the df_animals_subset
    df_animals_subset.loc[index, 'region_number'] = total_region_number
    df_animals_subset.loc[index, 'region_residue_number'] = total_region_residue_number

    n += 1
    print(n, sort_code, coding_seqment_number, total_protein_number, total_predicted_protein_number)

In [58]:
df_animals_subset['coding_segment_count'] = df_animals_subset['coding_segment_count'].astype(int)

df_animals_subset['protein_number'] = df_animals_subset['protein_number'].astype(int)
df_animals_subset['protein_residue_number'] = df_animals_subset['protein_residue_number'].astype(int)

df_animals_subset['predicted_protein_number'] = df_animals_subset['predicted_protein_number'].astype(int)
df_animals_subset['predicted_protein_residue_number'] = df_animals_subset['predicted_protein_residue_number'].astype(int)

df_animals_subset['peptide_number'] = df_animals_subset['peptide_number'].astype(int)
df_animals_subset['peptide_residue_number'] = df_animals_subset['peptide_residue_number'].astype(int)

df_animals_subset['region_number'] = df_animals_subset['region_number'].astype(int)
df_animals_subset['region_residue_number'] = df_animals_subset['region_residue_number'].astype(int)

/tmp/ipykernel_4155020/1343948450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_animals_subset['coding_segment_count'] = df_animals_subset['coding_segment_count'].astype(int)
/tmp/ipykernel_4155020/1343948450.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_animals_subset['protein_number'] = df_animals_subset['protein_number'].astype(int)
/tmp/ipykernel_4155020/1343948450.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [59]:
df_animals_subset

,Sort,Species,Genome composition,GENBANK accession,segment_count,coding_segment_count,protein_number,protein_residue_number,predicted_protein_number,predicted_protein_residue_number,peptide_number,peptide_residue_number,region_number,region_residue_number
0,12760,Ichnoviriform fumiferanae,dsDNA,AB289903|AB289904|AB289905|AB289906|AB289907|A...,105,74,103,20755,103,20755,0,0,0,0
1,12759,Ichnoviriform fugitivi,dsDNA,AY556383|AB291165|AB291166|AY935249|AB291167|A...,56,55,143,26717,143,26717,0,0,0,0
2,12723,Bracoviriform congregatae,dsDNA,AJ632304|AJ632305|AJ632306|AJ632307|AJ632308|A...,30,29,155,42830,155,42830,0,0,0,0
3,12729,Bracoviriform facetosae,dsDNA,EU001285|EU001284|EU001283|EU001282|EU001281|E...,29,7,109,22892,109,22892,0,0,0,0
4,12737,Bracoviriform indiense,dsDNA,EF051512|EF051511|EF051510|EF051509|EF051508|E...,27,6,27,7804,27,7804,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4402,8906,Ephemerovirus febris,ssRNA(-),AF234533,1,1,12,4809,11,2665,0,0,3,1970
4403,8907,Ephemerovirus guangdong,ssRNA(-),OK086698,1,1,10,4700,9,2584,0,0,3,1924
4404,8908,Ephemerovirus hayes,ssRNA(-),MH507506,1,1,10,4775,9,2670,0,0,3,1963
4405,8909,Ephemerovirus henan,ssRNA(-),OK086697,1,1,10,4693,9,2564,0,0,3,1951


In [60]:
df_animals_subset.to_csv("./VMR_MSL38_v2_acc_segmen_protein_peptide_region_info.csv", index=False)

In [61]:
df_animals_subset_short = df_animals_subset.drop(columns=['GENBANK accession'])

In [62]:
df_animals_subset_short.to_csv("./VMR_MSL38_v2_segmen_protein_peptide_region_info.csv", index=False)

In [71]:
df_animals_subset_short

,Sort,Species,Genome composition,segment_count,coding_segment_count,protein_number,protein_residue_number,predicted_protein_number,predicted_protein_residue_number,peptide_number,peptide_residue_number,region_number,region_residue_number
0,12760,Ichnoviriform fumiferanae,dsDNA,105,74,103,20755,103,20755,0,0,0,0
1,12759,Ichnoviriform fugitivi,dsDNA,56,55,143,26717,143,26717,0,0,0,0
2,12723,Bracoviriform congregatae,dsDNA,30,29,155,42830,155,42830,0,0,0,0
3,12729,Bracoviriform facetosae,dsDNA,29,7,109,22892,109,22892,0,0,0,0
4,12737,Bracoviriform indiense,dsDNA,27,6,27,7804,27,7804,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4402,8906,Ephemerovirus febris,ssRNA(-),1,1,12,4809,11,2665,0,0,3,1970
4403,8907,Ephemerovirus guangdong,ssRNA(-),1,1,10,4700,9,2584,0,0,3,1924
4404,8908,Ephemerovirus hayes,ssRNA(-),1,1,10,4775,9,2670,0,0,3,1963
4405,8909,Ephemerovirus henan,ssRNA(-),1,1,10,4693,9,2564,0,0,3,1951
